## PageRank
Pagerank is the algorithm which Google uses to rank it's search results. It uses the connectivity information of a graph to rank the importance of each vertex. See [Wikipedia](https://en.wikipedia.org/wiki/PageRank) for more details on the algorithm.

To compute the Pagerank scores for a graph in cuGraph we use:
**cugraph.pagerank(G,alpha=0.85, max_iter=100, tol=1.0e-5)**
* G: cugraph.Graph object
* alpha: float, The damping factor represents the probability to follow an outgoing edge.
* max_iter: int, The maximum number of iterations before an answer is returned. This can be used to limit the execution time and do an early exit before the solver reaches the convergence tolerance. If this value is lower or equal to 0 cuGraph will use the default value, which is 100
* tol: float, Set the tolerance the approximation, this parameter should be a small magnitude value. The lower the tolerance the better the approximation. If this value is 0.0f, cuGraph will use the default value which is 1.0E-5. Setting too small a tolerance can lead to non-convergence due to numerical roundoff. Usually values between 0.01 and 0.00001 are acceptable.

Returns a cudf.DataFrame object with two columns:
* df['vertex']: The vertex identifier for the vertex
* df['pagerank']: The pagerank score for the vertex

In [1]:
# Import needed libraries
import cugraph
import cudf
import numpy as np
from scipy.io import mmread

In [2]:
# Read in the data file into scipy matrix format
mmFile='/datasets/networks/karate.mtx'
M = mmread(mmFile).asfptype().tolil()
M = M.tocsr()

In [3]:
# Load the structure of the graph into GPU memory and create a CuGraph
# graph object:
row_offsets = cudf.Series(M.indptr)
col_indices = cudf.Series(M.indices)
values = cudf.Series(M.data)
G = cugraph.Graph()
G.add_adj_list(row_offsets, col_indices, values)

In [4]:
# Call cugraph.pagerank to get the pagerank scores
df = cugraph.pagerank(G)

In [6]:
# Find the most important vertex using the scores:
bestScore = df['pagerank'][0]
bestVert = df['vertex'][0]
for i in range(len(df)):
    if df['pagerank'][i] > bestScore:
        bestScore = df['pagerank'][i]
        bestVert = df['vertex'][i]
print("Best vertex is " + str(bestVert) + " with score of " + str(bestScore))

Best vertex is 33 with score of 0.10091735
